# TO DO'S

 - Data augmentation
     - shift
     - mirror?? (maybe not)
     - PCA? (why not)
 - reduce overfitting
 

In [12]:
import torch
import math

from torch import Tensor
from torch import optim
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch import nn

from matplotlib import pyplot as plt

from models import *

import numpy as np
# to load the data
import dlc_bci as bci

### test the imported data:

In [2]:
pow(2,3)

8

In [3]:
#CRIT = 'MSE'
CRIT = 'cross'

HD_SIGNAL = False

if HD_SIGNAL == True : SIGNAL_LENGTH = 500
elif HD_SIGNAL == False : SIGNAL_LENGTH = 50
    

train_input, train_target = bci.load ( root = './data_bci',train=True, download=False, one_khz=HD_SIGNAL)
print( str( type(train_input) ), train_input.size () )
print( str( type(train_target) ), train_target.size () )

test_input , test_target = bci.load ( root = './data_bci' , train = False, download=False, one_khz=HD_SIGNAL)
print( str( type(test_input) ) , test_input.size () )
print( str( type(test_target) ) , test_target.size () )

print('train --- mean {:0.2f}, std: {:0.2f}'.format(train_input.mean(),train_input.std()),\
     'target mean: ',train_target.numpy().mean())
print('test --- mean {:0.2f}, std: {:0.2f}'.format(test_input.mean(),test_input.std()),\
     'target mean: ',test_target.numpy().mean())


# transform into one-hot-encoding
def convert_to_one_hot(data):
    return np.eye(2)[data]

#one_hot_targets = np.eye(2)[train_target]
#train_target = one_hot_targets
if CRIT=='MSE':
    train_target = convert_to_one_hot(train_target)
    test_target = convert_to_one_hot(test_target)


# Convert them into Variables
train_input = Variable(Tensor(train_input))
test_input = Variable(test_input)
if CRIT == 'MSE':
    train_target = Variable(torch.Tensor(train_target))
    test_target = Variable(torch.Tensor(test_target))
elif CRIT == 'cross':
    train_target = Variable(torch.LongTensor(train_target))
    test_target = Variable(torch.LongTensor(test_target))

# normalise the data (note that the channels differ sometimes a lot)
def normalize_data(dataset):
    mu, std = dataset.mean(0), dataset.std(0) # normalisation is done component-wise
    return dataset.sub(mu).div(std)
    
#train_input = normalize_data(train_input)
#test_input = normalize_data(test_input)

print( str( type(train_input) ), train_input.size () )
print( str( type(train_target) ), train_target.size () )


<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
train --- mean 24.03, std: 41.49 target mean:  0.496835443038
test --- mean 21.69, std: 40.83 target mean:  0.51
<class 'torch.autograd.variable.Variable'> torch.Size([316, 28, 50])
<class 'torch.autograd.variable.Variable'> torch.Size([316])


### Model:

In [4]:
"""def create_model_1():
    return Net(nb_hidden=100, nb_init_filters = 16, nb_convs=2, kernel_size=5, length_signal = SIGNAL_LENGTH )"""

class Net_seq(nn.Module):

    def __init__(self, hidden = 100, num_classes=2, nl='relu',iterations = 0,length_signal = SIGNAL_LENGTH ):
        
        super().__init__()
        self.dropping_prob = 0.5
        if nl == 'leaky': f_non_linearity = nn.LeakyReLU(negative_slope=0.01,inplace=True)
        elif nl == 'tanh' : f_non_linearity =  nn.Tanh()
        elif nl == 'relu': f_non_linearity = nn.ReLU(inplace=True)
            
        filter_size = 3
        my_dropout = nn.Dropout(p=0.6)
        ################### layers ################
        
        f_layers = []
        f_layers.append(nn.Conv1d(28, 8, kernel_size=filter_size)) # 7 
        f_layers.append(f_non_linearity)
        
        f_layers.append(nn.Conv1d(8, 8, kernel_size=filter_size))
        f_layers.append(f_non_linearity)
        f_layers.append(nn.BatchNorm1d(8, affine=True))
        f_layers.append(my_dropout)
        #f_layers.append(nn.MaxPool1d(kernel_size=2))
        
        
        for i in range (iterations):
            f_layers.append(nn.Conv1d(4, 4, kernel_size=3))
            f_layers.append(f_non_linearity)
            f_layers.append(my_dropout)
        
        f_layers.append(nn.Conv1d(8, 16, kernel_size=filter_size))
        f_layers.append(f_non_linearity)
        f_layers.append(nn.BatchNorm1d(16, affine=True))
        f_layers.append(my_dropout)
        
        self.features = nn.Sequential(*f_layers)
        
        self.final_filters = length_signal - 3*(filter_size//2)*2 #-iterations*2
        self.final_length = 16
        
        c_layers = [] 
        c_layers.append(nn.Linear(self.final_filters*self.final_length, hidden))
        c_layers.append(nn.ReLU(inplace=True))
        #c_layers.append(nn.BatchNorm1d(hidden, eps=1e-05, momentum=0.1, affine=False))
        #c_layers.append(nn.Dropout(inplace=True))
        """c_layers.append(nn.Linear(hidden, hidden))
        c_layers.append(nn.ReLU(inplace=True))
        c_layers.append(my_dropout)
        c_layers.append(nn.Linear(hidden, hidden))
        c_layers.append(nn.ReLU(inplace=True))"""
        c_layers.append(nn.Linear(hidden, num_classes))
        #c_layers.append(nn.Softmax(dim=1))
        
        self.classifier = nn.Sequential(* c_layers)
        
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.final_filters*self.final_length)
        x = self.classifier(x)
        return x
        

# 14% with just 2 conv layers 26,16 filter 5 ;; 16,32, filter 5, 2 fcl with hidden = 100

def create_net_seq():
    return Net_seq()

### Parameters: 

In [9]:
mini_batch_size = 70
epochs = 400

eta = 0.008

if CRIT == 'MSE':
    criterion = nn.MSELoss()
elif CRIT == 'cross':
    criterion = nn.CrossEntropyLoss()

#torch.norm()

### Helper Functions:

In [10]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size = mini_batch_size):
    nb_errors = 0
    for i in range(0, data_input.size(0)):
        model.train(False)
        if model.training == False:
            output = model.forward(data_input[i].view(1,28,SIGNAL_LENGTH))
        else : print('DROPING OUT IN TESTING')
        #print(output)
        _, predicted = torch.max(output.data, 1)                            
        true_val = data_target.data[i]
        if CRIT == 'MSE':
            true_val = true_val[0]
        predicted = predicted[0]
        #print(true_val)
        #print(predicted)
        if (true_val!= predicted):
            nb_errors += 1
        #print('LALALALALALA')
    return nb_errors

In [16]:
def train_model(model, train_input, train_target, mini_batch_size):
    
    #optimizer = torch.optim.SGD(model.parameters(), lr = eta)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0)
    #optimizer = torch.optim.LBFGS(model.parameters(), lr=1, max_iter=20, max_eval=None, tolerance_grad=1e-05)
    #optimizer = torch.optim.SparseAdam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
    
    train_list = []
    test_list = []
    for e in range(epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            model.train(True)
            if model.training == True:
                output = model.forward(train_input[b:b+mini_batch_size])
            else : print('NOT TRAINING')
                
            loss = criterion(output, train_target[b:b+mini_batch_size])
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
            """# for L1 normalisation
            for p in model.parameters():
                p.data -= p.data.sign() * p.data.abs().clamp(max = 0.001)"""

        if (e%10==0):
            train_errors = compute_nb_errors(model, data_input=train_input, data_target=train_target)
            test_errors = compute_nb_errors(model, data_input=test_input, data_target=test_target)
            train_list.append(train_errors)
            test_list.append(test_errors)
            print('epoch #',e,\
                  'Train: {:0.2f}%'.format( train_errors/train_input.size(0)*100),\
                  'Test:  {:0.2f}%'.format(test_errors / test_input.size(0)*100),\
                  
            )
            #print('Loss:  {:0.5f} '.format(loss.data[0]))
    return train_list, test_list

            

# Testing Zone:

In [17]:
#model_list = [create_model_1, create_net_seq]
for m in [create_net_seq]:
    print(m.__name__)
    model = m()
    train_list, test_list = train_model(model, train_input, train_target, mini_batch_size)

create_net_seq
epoch # 0 Train: 50.32% Test:  50.00%
epoch # 10 Train: 43.35% Test:  54.00%
epoch # 20 Train: 42.72% Test:  43.00%
epoch # 30 Train: 37.97% Test:  47.00%
epoch # 40 Train: 36.08% Test:  50.00%
epoch # 50 Train: 35.44% Test:  47.00%
epoch # 60 Train: 34.81% Test:  47.00%
epoch # 70 Train: 32.28% Test:  51.00%
epoch # 80 Train: 30.38% Test:  53.00%
epoch # 90 Train: 31.65% Test:  52.00%
epoch # 100 Train: 29.11% Test:  54.00%
epoch # 110 Train: 29.75% Test:  49.00%
epoch # 120 Train: 26.27% Test:  48.00%
epoch # 130 Train: 26.58% Test:  48.00%
epoch # 140 Train: 23.73% Test:  49.00%
epoch # 150 Train: 24.68% Test:  50.00%
epoch # 160 Train: 22.47% Test:  50.00%
epoch # 170 Train: 20.89% Test:  47.00%
epoch # 180 Train: 19.94% Test:  48.00%
epoch # 190 Train: 18.67% Test:  47.00%
epoch # 200 Train: 18.35% Test:  46.00%
epoch # 210 Train: 15.82% Test:  45.00%
epoch # 220 Train: 15.51% Test:  47.00%
epoch # 230 Train: 16.46% Test:  46.00%
epoch # 240 Train: 13.92% Test:  43.

In [20]:
plt.plot(train_list/train_input.size(0)*100)
plt.plot(test_list/ test_input.size(0)*100)
plt.show()

TypeError: unsupported operand type(s) for /: 'list' and 'int'